*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 1: Trace and Low-Degree Extension

- [Video Lecture (youtube)](https://www.youtube.com/watch?v=Y0uJz9VL3Fo)
- [Slides (PDF)](https://starkware.co/wp-content/uploads/2021/12/STARK101-Part1.pdf)

Today we will develop a STARK prover for the FibonacciSq sequence over a finite field.
The FibonacciSq sequence is defined by the recurrence relation $a_{n+2} = a_{n+1} ^2 + a_n ^2$.
<br>By the end of the day, your code will produce a *STARK* proof attesting to the following statement: <br>**I know a field element $X\in \mathbb{F}$ such that the 1023rd element of the FibonacciSq sequence starting with $1, X$ is $2338775057$**.
<br><br>
## The Basics
### FieldElement class
We use our `FieldElement` class to represent field elements.<br> You can construct instances of `FieldElement` from integers, and then add, multiply, divide, get inverse, and so on.
The underlying field of this class is $\mathbb{F}_{3221225473}$ ($3221225473 = 3 \cdot 2^{30} + 1$), so all operations are done modulo 3221225473.
<br><br>
Try it by running the following cell (shift + enter):

In [2]:
from field import FieldElement
FieldElement(1610612736)
"1610612736 ==> 1610612736"
"1610612737 ==> -1610612736"

'1610612737 ==> -1610612736'

## FibonacciSq Trace

To start, let's construct a list `a` of length 1023, whose first two elements will be FieldElement objects representing 1 and 3141592, respectively. The next 1021 elements will be the FibonacciSq sequence induced by these two elements. `a` is called the trace of FibonacciSq, or, when the context is clear, the trace. <br>
Correct the code below to fill `a`:

In [7]:
a = [FieldElement(1), FieldElement(3141592)] # we already have the first two elements

#we need to add the next elements of the fibSequence 
for i in range(1021):
    a.append(a[-2]**2 + a[-1]**2)
print(len(a))

1023


Solution (click to the ... to unhide):

In [6]:
a = [FieldElement(1), FieldElement(3141592)]
while len(a) < 1023:
    a.append(a[-2] * a[-2] + a[-1] * a[-1])
    
print(len(a))

1023


### Test Your Code
Run the next cell to test that you have filled `a` correctly.<br> Note that this is in fact a verifier, albeit very naive and non-succinct one, as it goes over the sequence, element by element, making sure it is correct.

In [8]:
assert len(a) == 1023, 'The trace must consist of exactly 1023 elements.'
assert a[0] == FieldElement(1), 'The first element in the trace must be the unit element.'
for i in range(2, 1023):
    assert a[i] == a[i - 1] * a[i - 1] + a[i - 2] * a[i - 2], f'The FibonacciSq recursion rule does not apply for index {i}'
assert a[1022] == FieldElement(2338775057), 'Wrong last element!'
print('Success!')

Success!


## Thinking of Polynomials
We now want to think of the sequence as the evaluation of some, yet unknown, polynomial $f$ of degree 1022 (due to the Unisolvence Theorem).
We will choose the domain to be some subgroup $G \subseteq \mathbb{F}^\times$ of size 1024, for reasons that will become clear later.

(Recall that $\mathbb{F}^\times$ denotes the multiplicative group of $\mathbb{F}$, which we get from $\mathbb{F}$ by omitting the zero element with the induced multiplication from the field. A subgroup of size 1024 exists because $\mathbb{F}^\times$ is a cyclic group of size $3\cdot 2^{30}$, so it contains a subgroup of size $2^i$ for any $0 \leq i \leq 30$).
### Find a Group of Size 1024
If we find an element $g \in \mathbb{F}$ whose (multiplicative) order is 1024, then $g$ will generate such a group.
The class `FieldElement` provides a static method `generator()` which returns an element that generates $\mathbb{F}^\times$ (whose order is $|\mathbb{F}^\times|$).
1. Use it to obtain a generator $g$ for $G$.
2. Create a list called `G` with all the elements of $G$, such that $G[i] := g^i$.

*Hint: When $k$ divides $|\mathbb{F}^\times|$, $g^k$ generates a group of size $\frac {|\mathbb{F}^\times|}{k}$, and the n-th power of some `FieldElement` $x$ can be computed by calling `x ** n `.*


The FieldElement.generator() method returns a field element that is a generator of the field, which means that it generates every element in the field when raised to different powers. In this case, g is set to the generator raised to the power of 3 * 2 ** 20.

When computing the value of g by doing the calculation FieldElement.generator() ** (3 * 2 ** 20), we are actually computing the value of 5 ** (3 * 2 ** 20) modulo 3 * 2**30 + 1.

As the exponent is very large, the result is the remainder of the division of the value of 5 raised to the 3 * 2 ** 20 by 3 * 2 ** 30 + 1. Because the modulus is prime, we know that the result will be a unique number between 0 and k_modulus-1 which is -1365964089


In [49]:
# Change the following line so that g will generate a group of size 1024
# this solution makes sense too.
g = FieldElement.generator()
print(g)
# Fill G with the elements of G such that G[i] := g ** i
G = [] 
G = [g ** i for i in range(1024)]

5


Solution:

In [56]:
#todo: check if g = FieldElement.generator() ** (3 * 2 ** 20)//1024 is solution too.

g = FieldElement.generator() ** (3 * 2 ** 20)
print(g)
G = [g ** i for i in range(1024)]

-1365964089


Run the next cell to test your code. 

In [47]:
# Checks that g and G are correct.
assert g.is_order(1024), 'The generator g is of wrong order.'
b = FieldElement(1)
for i in range(1023):
    assert b == G[i], 'The i-th place in G is not equal to the i-th power of g.'
    b = b * g
    assert b != FieldElement(1), f'g is of order {i + 1}'
    
if b * g == FieldElement(1):
    print('Success!')
else:
    print('g is of order > 1024')

Success!


### Polynomial class
We provide you with a class called `Polynomial`. The simplest way to construct a `Polynomial` is by using the variable `X` (note that it's a capital `X`) which represents the formal variable $x$:

In [57]:
from polynomial import X
# The polynomial 2x^2 + 1.
p = 2*X**2 + 1
# Evaluate p at 2:
print(p(2))
# Type a polynomial's name, on its own, in the last line of a cell to display it
p

9


### Interpolating a Polynomial
Our `polynomial` module provides a Lagrange interpolation function, whose arguments are:
* x_values: x-values of G that the polynomial's values for them is known. [List]
* y_values: the corresponding y-values. [List]

It returns the unique `Polynomial` of degree < `len(x_values)` instance that evaluates to `y_values[i]` on `x_values[i]` for all i.

Lagrange interpolation is a method for finding a polynomial that passes through a given set of points.

In this case, the function calculate_lagrange_polynomials takes in two arguments, x_values and y_values, which are lists of x-values and y-values respectively. These lists are used to find a polynomial that passes through all the given points.

For example, let's say we have the following set of points:
(1,2), (2,3), (3,4)

We can use the Lagrange interpolation function to find a polynomial that passes through all these points.

x_values = [1,2,3]
y_values = [2,3,4]

polynomial = calculate_lagrange_polynomials(x_values, y_values)

The returned polynomial would be of the form: f(x) = ax^2 + bx + c, where a, b, and c are coefficients that are calculated based on the given x-values and y-values.

We can then evaluate the polynomial at any point within the domain of the x-values and it would give the corresponding y-value. For example, if we evaluate the polynomial at x = 2, it would give y = 3.

In this way, the function calculate_lagrange_polynomials finds a unique polynomial of degree < len(x_values) that evaluates to y_values[i] on x_values[i] for all i.

the exponent of the polynomial is 2, is this related to that the number of inputs is 3? 

Yes, the degree of the polynomial is related to the number of input points used in the Lagrange interpolation.

When we have n input points, we need a polynomial of degree n-1 to pass through all of them. This is because a polynomial of degree n-1 has n coefficients that can be adjusted to fit the n input points.

For example, in the case you've provided in your example, the polynomial has an exponent of 2 because we have 3 input points. A polynomial of degree 2 (i.e. a quadratic polynomial) has 3 coefficients that can be adjusted to fit the 3 input points.

This is why the unique polynomial of degree < len(x_values) is returned because the degree of the polynomial must be one less than the number of input points.


Run the following cell to get help on the function `interpolate_poly`.

In [58]:
from polynomial import interpolate_poly
interpolate_poly?

Signature: interpolate_poly(x_values, y_values)
Docstring:
Returns a polynomial of degree < len(x_values) that evaluates to y_values[i] on x_values[i] for
all i.
File:      c:\users\henry alberto\desktop\stark_prover\stark101\tutorial\polynomial.py
Type:      function

Suppose that `a` contains the values of some polynomial over `G` (except for `G[-1]`, since `a` is one element shorter).
Use `interpolate_poly()` to get `f` and get its value at `FieldElement(2)`. 

In [ ]:
from polynomial import interpolate_poly
# Fix the following so that v will contain the value of f at FieldElement(2)
# Note that interpolate_poly may take up to a minute to run.
f = interpolate_poly(G[:-1], a)
v = f(2)

Solution:

In [66]:
f = interpolate_poly(G[:-1], a)
v = f(2)
print(v)

1302089273


Run test:

In [63]:
assert v == FieldElement(1302089273)
print('Success!')

Success!


## Evaluating on a Larger Domain
The sequence or trace, viewed as evaluations of a polynomial $f$ on $G$, can now be extended by evaluating $f$ over a larger domain, thereby creating a Reed-Solomon error correction code.

### Cosets
To that end, we must decide on a larger domain on which $f$ will be evaluated. 
We will work with a domain that is 8 times larger than $G$. <br>A natural choice for such a domain is to take some group $H$ of size 8192 (which exists because 8192 divides $|\mathbb{F}^\times|$), and shift it by the generator of $\mathbb{F}^\times$, thereby obtaining a [coset](https://en.wikipedia.org/wiki/Coset) of $H$.

Create a list called `H` of the elements of $H$, and multiply each of them by the generator of $\mathbb{F}^\times$ to obtain a list called `eval_domain`. In other words, eval_domain = $\{w\cdot h^i | 0 \leq i <8192  \}$ for $h$ the generator of $H$ and $w$ the generator of $\mathbb{F}^\times$.

Hint: You already know how to obtain $H$ - similarly to the way we got $G$ a few minutes ago.


In [4]:
# Fix the following, make sure that the the element of h are powers of its generator in 
# order, that is - H[0] will be the unit, H[1] will be h (H's generator), H[2] will be H's
# generator squared, etc.
w = FieldElement.generator()
h = w **((2**30*3)/8192)
print(h)
H = [h**i for i in range(8192)]
eval_domain = [w*x for x in H]

-1486748106


Solution:

In [68]:
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 8192)
print(h)
H = [h ** i for i in range(8192)]
print(H[0],H[1],H[2])
eval_domain = [w * x for x in H]


-1486748106
1 -1486748106 211283056


Run test:

The first line checks that all elements in the eval_domain list are unique. The set data structure only keeps unique elements, so if the length of the set of eval_domain is equal to the length of the eval_domain list, it means that there are no duplicate elements in the list.

For example:

eval_domain = [1, 2, 3, 4, 5]
assert len(set(eval_domain)) == len(eval_domain) # this will pass

eval_domain = [1, 2, 3, 4, 5, 3]
assert len(set(eval_domain)) == len(eval_domain) # this will fail, because 3 is repeated

The second assert statement is checking that the element at the index 1 of the list H is equal to the generator h of the group H. The for loop is checking that each element of the list eval_domain is equal to (h*w)^i where w is the generator of the field and i is the index of the element in the list eval_domain. And if all the assert statements are true, the code will print 'Success!'.

This assertion (3rd) is checking that the element at index i of the eval_domain list is equivalent to (w_inv * eval_domain[1]) raised to the power of i and then multiplied by w. This is a way to check that all the elements of eval_domain are in the coset of H shifted by the generator of the field element. If this assert statement passes for all i, then it means that the eval_domain list is indeed a coset of H shifted by the generator of the field element as desired.

In [69]:
from hashlib import sha256
assert len(set(eval_domain)) == len(eval_domain)
w = FieldElement.generator()
w_inv = w.inverse()
assert '55fe9505f35b6d77660537f6541d441ec1bd919d03901210384c6aa1da2682ce' == sha256(str(H[1]).encode()).hexdigest(),\
    'H list is incorrect. H[1] should be h (i.e., the generator of H).'
for i in range(8192):
    assert ((w_inv * eval_domain[1]) ** i) * w == eval_domain[i]
print('Success!')

Success!


### Evaluate on a Coset
Time to use `interpolate_poly` and `Polynomial.poly` to evaluate over the coset. Note that it is implemented fairely naively in our Python module, so interpolation may take up to a minute.<br>
Indeed - interpolating and evaluating the trace polynomial is one of the most computationally-intensive steps in the STARK protocol, even when using more efficient methods (e.g. FFT).

In [ ]:
# Fill f_eval with the evaluations of f on eval_domain.
f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

Solution:

In [70]:
f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

Run test:

This code is testing the output of the f_eval variable, which is the result of evaluating the polynomial f on the eval_domain using the interpolate_poly function. It uses the built-in python library hashlib and specifically the sha256 function to compute a hash of the serialized f_eval variable. Then it compares this computed hash to a precomputed hash value, '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb'. If the computed hash is equal to the precomputed hash, it will print "Success!" indicating that the output is correct. If the computed hash is not equal to the precomputed hash, it will raise an assert exception.

An example of how this code might be used is as follows:
Let's say we have a function interpolate_poly(x_values, y_values) which takes in 2 lists as inputs, x_values and y_values, and returns a polynomial that goes through all the given points, and then we have a list G which is the group of size 1024 and a list a of y values.

G = [FieldElement(x) for x in range(1024)]
a = [FieldElement(x*x) for x in range(1024)]

Now we want to use this interpolate_poly function to find the polynomial that goes through the points given by G[:-1] and a and evaluate this polynomial on the eval_domain which we created previously.

f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

Now, this code will take this f_eval variable, serialize it and calculate the sha256 hash of the serialized f_eval variable and compare it with the precomputed hash value, '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb'. If the calculated hash and the precomputed hash are equal, it will print 'Success!' otherwise it will raise an assert exception.

In [71]:
# Test against a precomputed hash.
from hashlib import sha256
from channel import serialize
assert '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb' == sha256(serialize(f_eval).encode()).hexdigest()
print('Success!')

Success!


## Commitments
We will use [Sha256](https://en.wikipedia.org/wiki/SHA-2)-based [Merkle Trees](https://en.wikipedia.org/wiki/Merkle_tree) as our commitment scheme.
A simple implementation of it is available to you in the `MerkleTree` class.
Run the next cell (for the sake of this tutorial, this also serves as a test for correctness of the entire computation so far):



In [72]:
from merkle import MerkleTree
f_merkle = MerkleTree(f_eval)
assert f_merkle.root == '6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04'
print('Success!')

Success!


## Channel
Theoretically, a STARK proof system is a protocol for interaction between two parties - a prover and a verifier. In practice, we convert this interactive protocol into a non-interactive proof using the [Fiat-Shamir Heuristic](https://en.wikipedia.org/wiki/Fiat%E2%80%93Shamir_heuristic). In this tutorial you will use the `Channel` class, which implements this transformation. This channel replaces the verifier in the sense that the prover (which you are writing) will send data, and receive random numbers or random `FieldElement` instances.

This simple piece of code instantiates a channel object, sends the root of your Merkle Tree to it. 
Later, the channel object can be called to provide random numbers or random field elements.

In [73]:
from channel import Channel
channel = Channel()
channel.send(f_merkle.root)

Lastly - you can retrieve the proof-so-far (i.e., everything that was passed in the channel up until a certain point) by printing the member `Channel.proof`.

In [74]:
print(channel.proof)

['send:6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04']
